# Raspberry Pi output
Here we will look at an introduction how the raspberry PI GPIO pins can be used to trun on/off external components.

Start by connecting the 5V of the breakout board to the red rail of the breadboard. And the ground of the breakout board to the blue rail. Also connect the upper and lower parts of the rail to to each other. We do this so that it is simple to connect new components to 5V and GND

IMAGE


Now connect a cable to GPIO pin 18 (G18 on the breakout board) and to anywhre on the board (orange). Then connect an LED and a 220 OHM resitor in series to G18 and to ground.

IMAGE

In [ ]:
import RPi.GPIO as GPIO
import time
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

GPIO.output(18,GPIO.HIGH)
time.sleep(2)
GPIO.output(18,GPIO.LOW)

GPIO.cleanup() 

Now we have a good indicator that G18 is on. Let's try to control a larger current using G18 and a transistor.

Instead of the LED and resitor going to ground, connect them to the base (middle pin) of a transitor. Connect the emittor (left pin) of the transistor to ground. In this way, when G18 is on, the transistor allows current to flow from the collector to ground.

IMAGE

Now connect a 220 Ohm resistor and a Blue LED in series to the collector and 5V.

IMAGE

Run the same code as before


In [ ]:
import RPi.GPIO as GPIO
import time
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)

GPIO.output(18,GPIO.HIGH)
time.sleep(2)
GPIO.output(18,GPIO.LOW)

GPIO.cleanup() 

We are now using G18, which is 3.3V to drive a larger potential of 5V to power the blue LED

G18 can only be on and off. 3.3V or 0.0V. But G18 has a special feature called Pulse Width Modulation (PWM), where in a time span of 1 second it is on/off a certain number of times. And we can vary the width of the ON pulses to vary the power output of G18. We can use this to vary the brightness the LEDs.

In [ ]:
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)
pi_pwm = GPIO.PWM(18,1000)#create PWM instance with frequency
pi_pwm.start(0)#start PWM of required Duty Cycle

pi_pwm.ChangeDutyCycle(100)
time.sleep(1)
pi_pwm.ChangeDutyCycle(75)
time.sleep(1)
pi_pwm.ChangeDutyCycle(50)
time.sleep(1)
pi_pwm.ChangeDutyCycle(25)
time.sleep(1)
pi_pwm.ChangeDutyCycle(10)
time.sleep(1)
pi_pwm.ChangeDutyCycle(0)
time.sleep(1)
pi_pwm.ChangeDutyCycle(10)
time.sleep(1)
pi_pwm.ChangeDutyCycle(25)
time.sleep(1)
pi_pwm.ChangeDutyCycle(50)
time.sleep(1)
pi_pwm.ChangeDutyCycle(75)
time.sleep(1)
pi_pwm.ChangeDutyCycle(100)
time.sleep(1)

pi_pwm.stop()
GPIO.cleanup() 

EXERCIES: Any ideas how this can be useful?

# ADC and voltage dividers

Next we want to create a voltage divider between 5V and ground and measure it's voltage

On the other half of the breadboard, connect a wire to 5V and out some distance on the breadboard

IMAGE

In series to the 5V, conenct a 91 kOhm ressistor, a NTC thermistor, and ground. Keep the long legs of the NTC, you  will need them later. Connect an open ended cable (probe) where the resistor and NTC are connected togehter.

IMAGE

**Always use probes like this, if you slip with the probes of the voltmeter you can physically destry the Raspberry Pi. IF you accidentally conec 5V and 3.3V the raspberry Pi dies instantly.**

EXERCIES: Calculate the voltage at the probe.\


Connect the probe to A0 on the raspberry pi hat

IMAGE

In [ ]:
import i2c_module
import adc_module

adc = adc_module.ADC()
channel=0
millli_volts = adc.read_voltage(channel)
volts = millli_volts/1000
print(volts)

EXERCIES: why is it dfferent from the calculated value?\
EXERCIES: The ADC is 12-bit and can measure between 0 and 3.3 volts. What resistance should the 91 kOhm resistor have to maximize the range?, i,e to have 1.65 Volts over the NTC?

## Measuring temperature


To calcualte the temperature of the NTC we follow a mathematical function given by the manufacturer. This function requires some parameters.

In [ ]:
channel = 0
R0 = 10000
b = 3936
Rfx = 91000
voltage = 5.2
adc = adc_module.ADC()

deltav=voltage
Vin=adc.read_voltage(channel)/1000
To=298.0  #To of NTC
Ri=int(R0)*math.exp(-int(b)/To)
#Rn=(Vin*int(self.__Rfx))/((deltav*2)-Vin) # BNL version
Rn=(Vin*int(Rfx))/((deltav)-Vin)
T=int(b)/(math.log(Rn/Ri))-273

print(T)

This is very messy to write each time we want a temperature, so therefore we create a tmperature sensor calss, which has a function "read_temperature". In our script, we can then create as sensor object, and call the read_temperature function of this object each time we wnat a temperature reading

In [12]:
import math
import i2c_module
import adc_module

class NTC:
    def __init__(self,channel,voltage):
        self.channel = channel
        self.R0 = 10000
        self.b = 3936
        self.Rfx = 91000
        self.voltage = voltage
        self.adc = adc_module.ADC()

    def read_temperature(self):
        deltav=self.voltage
        Vin=self.adc.read_voltage(self.channel)/1000
        To=298.0  #To of NTC
        Ri=int(self.R0)*math.exp(-int(self.b)/To)
        #Rn=(Vin*int(self.__Rfx))/((deltav*2)-Vin) # BNL version
        Rn=(Vin*int(self.Rfx))/((deltav)-Vin)
        T=int(self.b)/(math.log(Rn/Ri))-273
        return round(T,2)

After the class is defined, we create an object of that class, and assign it to avariable name

In [13]:
sensor_1 = NTC(0,5.21)

We can then very easily read the temperature in one line

In [14]:
print(sensor_1.read_temperature())

22.03


We can also read out the temperature continiously in a for loop. Click the stop symbol at the top to stop it.

In [23]:
import time
for i in range(0,60):
    print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')
    time.sleep(1)

KeyboardInterrupt: 

**Unplugg the 5V of the votlage divider**. Now bend the NTC so that it touchse the resistor of the blue ntc. make sure that the legs of the NTC are not touchin each other, this would short circuit the NTC. Re-insert the 5V What happens to the tmeperature when G18 is on?

In [31]:
import RPi.GPIO as GPIO

GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)
pi_pwm = GPIO.PWM(18,1000)#create PWM instance with frequency
pi_pwm.start(0)#start PWM of required Duty Cycle

print(f"Initial temperature: {sensor_1.read_temperature()}")

pi_pwm.ChangeDutyCycle(100)

try:
    for i in range(0,60):
        print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')
        time.sleep(1)

except KeyboardInterrupt:
    print(f"Final temperature: {sensor_1.read_temperature()}")
    pi_pwm.stop()
    sys.exit()

print(f"Final temperature: {sensor_1.read_temperature()}")
pi_pwm.stop()
GPIO.cleanup() 

Initial temperature: 23.08
Final temperature: 23.99


Unplugg the 5V to the blue LED. Replace the blue LED and its resistor with a big 55 Ohm resistor. Re-connect the 5V, now to the 55 Ohm reisstor.

IMAGE

In [37]:
import RPi.GPIO as GPIO

GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(18,GPIO.OUT)
pi_pwm = GPIO.PWM(18,1000)#create PWM instance with frequency
pi_pwm.start(0)#start PWM of required Duty Cycle

print(f"Initial temperature: {sensor_1.read_temperature()}")

pi_pwm.ChangeDutyCycle(100)

try:
    for i in range(0,20):
        print('Temperature', f"{sensor_1.read_temperature()} C", end='\r')
        time.sleep(1)

except KeyboardInterrupt:
    print(f"Final temperature: {sensor_1.read_temperature()}")
    pi_pwm.stop()
    sys.exit()

print(f"Final temperature: {sensor_1.read_temperature()}")
pi_pwm.stop()
GPIO.cleanup() 

Initial temperature: 27.44
Final temperature: 33.84
